In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing

files.upload()

In [ ]:
df = pd.read_csv("healthcare-dataset-stroke-data.csv")
sns.set(rc={'figure.figsize':(20,9)})
df = df.dropna()
df = df.loc[df["age"] > 18]

sns.barplot(data=df,y = df.groupby("gender")["stroke"].transform("count"),x = "gender")
sns.barplot(data=df,y = df.groupby("age")["stroke"].transform("count"),x="age")
sns.scatterplot(data=df,x="age",y="bmi",hue="stroke")
sns.scatterplot(data=df,x="age",y="avg_glucose_level",hue="stroke")

be = preprocessing.LabelBinarizer()
le = preprocessing.LabelEncoder()
#male is 0 female is 1
df["gender"] = be.fit_transform(df["gender"])
# 0 is no 1 is yes
df["ever_married"] = be.fit_transform(df["ever_married"])
#0 is rural 1 is urban
df["Residence_type"] = be.fit_transform(df["Residence_type"])
# private is 2 govt_job is 0 self-employed is 3 1 is never_worked
df["work_type"] = le.fit_transform(df["work_type"])
#1 is formerly smoked 2 is never smoked 3 is smokes 4 0 is unknown
df["smoking_status"] = le.fit_transform(df["smoking_status"])

print(df["age"].describe())





In [ ]:
sns.scatterplot(data=df,x="age",y="avg_glucose_level",hue="stroke")